## 1. Instalando o Faiss

In [11]:
#!pip install --trusted-host pypi.org --trusted-host files.pythonhosted.org faiss-cpu
#!pip install faiss-gpu

# CPU-only version
#$ conda install -c pytorch faiss-cpu=1.8.0 --verbose -y

# GPU(+CPU) version
#$ conda install -c pytorch -c nvidia faiss-gpu=1.8.0

# GPU(+CPU) version with NVIDIA RAFT
#$ conda install -c pytorch -c nvidia -c rapidsai -c conda-forge faiss-gpu-raft=1.8.0

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for faiss-cpu from https://files.pythonhosted.org/packages/d3/ad/63eb31be05c38445781caecbc6d3b9dec151012194b00573ba34f29d5cf5/faiss_cpu-1.8.0-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/14.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.5 MB ? eta -:--:--
   ---------------------------------------- 0.1/14.5 MB 975.2 kB/s eta 0:00:15
   - -------------------------------------- 0.4/14.5 MB 3.8 MB/s eta 0:00:04
   -- ------------------------------------- 0.9/14.5 MB 6.2 MB/s eta 0:00:03
   --- ------------------------------------ 1.4/14.5 MB 7.2 MB/s eta 0:00:02
   ---- ----------------------------------- 1.8/14.5 MB 8.0 MB/s eta 0:00:02
   ------ --------------------------------- 2.3/14.5 MB 8.5 MB/s eta 0:00:02
   ------- -------------------------------- 2.8/14.5 MB 9.2 MB/s eta 0:00:02
   --------- ----------------

## 2. Criando os vetores com os dados

In [2]:
import numpy as np

# d é o tamanho dos vetores (embeddings)
d = 64                           # dimension

# nb é a quantidade de vetores indexados no banco de dados
nb = 100000                      # database size

# nq é o número de queries
nq = 10000                       # nb of queries

np.random.seed(1234)             # make reproducible

# xb é a matriz que contem todos os vetores que serão indexados no banco. Tamanho nb x d.
xb = np.random.random((nb, d)).astype('float32')
xb[:, 0] += np.arange(nb) / 1000.

# xq é a matriz que contém todos os vetores de queries. Tamanho nq x d
xq = np.random.random((nq, d)).astype('float32')
xq[:, 0] += np.arange(nq) / 1000.

## 3. Construindo um índice e adicionando os vetores a ele

In [12]:
import faiss                   # make faiss available

#Cria índice com versão mais simples que apenas realiza uma busca exaustiva de distância L2
index = faiss.IndexFlatL2(d)   # build the index

# Os índices precisam saber qual é a dimensionalidade dos vetores e a maioria requer uma fase de treinamento
# Para o IndexFlatL2, podemos pular essa operação

# Verifica se o índice já está treinado
print(index.is_trained)

# Adiciona os vetores ao índice do banco de dados
index.add(xb)                  # add vectors to the index

# Verifica o número de vetores indexados
print(index.ntotal)

True
100000


## 4. Buscando

In [13]:
# k é o número de vetores que serão retornados na busca
k = 4                          # we want to see 4 nearest neighbors

# teste de sanidade do banco
D, I = index.search(xb[:5], k) # sanity check
print(I)
print(D)

# busca propriamente dita
D, I = index.search(xq, k)     # actual search
print(I[:5])                   # neighbors of the 5 first queries
print(I[-5:])                  # neighbors of the 5 last queries

# I é uma matriz inteira de tamanho nq x k, onde a linha i contém os IDs dos k vizinhos do vetor de consulta i,
# ordenados por distância crescente

# D é uma matriz de ponto flutuante nq x k com as distâncias quadráticas correspondentes

[[  0 393 363  78]
 [  1 555 277 364]
 [  2 304 101  13]
 [  3 173  18 182]
 [  4 288 370 531]]
[[0.        7.1751733 7.2076297 7.2511625]
 [0.        6.3235645 6.684581  6.799946 ]
 [0.        5.7964087 6.391736  7.2815123]
 [0.        7.2779055 7.5279875 7.662846 ]
 [0.        6.7638035 7.2951202 7.368815 ]]
[[ 381  207  210  477]
 [ 526  911  142   72]
 [ 838  527 1290  425]
 [ 196  184  164  359]
 [ 526  377  120  425]]
[[ 9900 10500  9309  9831]
 [11055 10895 10812 11321]
 [11353 11103 10164  9787]
 [10571 10664 10632  9638]
 [ 9628  9554 10036  9582]]


Para realizar uma pesquisa semântica com termos de pesquisa em uma base de jurisprudência do TCU, você provavelmente deseja utilizar um tipo de índice que seja eficiente para calcular similaridades entre os vetores que representam os documentos da jurisprudência e os vetores que representam os termos de pesquisa.

O Faiss oferece vários tipos de índices que podem ser adequados para essa tarefa, dependendo das características específicas do seu conjunto de dados e das necessidades da sua aplicação. Aqui estão algumas sugestões:

IndexFlatL2: Este é o tipo mais simples de índice e pode ser uma escolha adequada se você estiver lidando com um conjunto de dados relativamente pequeno e a eficiência não for uma preocupação principal. No entanto, se você estiver trabalhando com uma base de jurisprudência grande, o IndexFlatL2 pode não ser a melhor escolha devido à sua complexidade de busca linear.

IndexIVFFlat: Este é um tipo de índice que divide o espaço de vetores em células usando quantização vetorial e usa a busca aproximada dentro de cada célula para acelerar a busca. Pode ser uma boa opção se você estiver lidando com um conjunto de dados grande e quiser equilibrar eficiência e precisão.

IndexHNSWFlat: Este é um tipo de índice que constrói uma estrutura de vizinhança hierárquica esparsa (HNSW) para acelerar a busca de vizinhos mais próximos. Pode ser uma escolha adequada se a eficiência de busca for uma prioridade e você estiver disposto a sacrificar um pouco de precisão em troca de velocidade.

IndexIDMap + IndexLSH: Esta é uma combinação de índices onde o IndexIDMap armazena os vetores e seus IDs correspondentes, enquanto o IndexLSH é usado para acelerar a busca aproximada. Pode ser uma boa opção se você estiver lidando com um conjunto de dados grande e quiser uma solução simples e eficiente.

IndexPQ: Este é um tipo de índice que usa quantização de produto para acelerar a busca aproximada. Pode ser uma escolha adequada se você estiver lidando com um conjunto de dados grande e quiser uma solução eficiente em termos de memória e tempo de busca.

A escolha do tipo de índice depende de vários fatores, como o tamanho do conjunto de dados, os requisitos de eficiência de busca e a precisão desejada. Você pode experimentar diferentes tipos de índices e ajustar os parâmetros conforme necessário para encontrar a melhor solução para o seu caso de uso específico.